<a href="https://colab.research.google.com/github/wingated/cs473/blob/main/labs/cs473_lab_week_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><p><b>After clicking the "Open in Colab" link, copy the notebook to your own Google Drive before getting started, or it will not save your work</b></p>

# BYU CS 473 Lab Week 2

## Introduction:
Welcome to your first lab for CS 473, Advanced Machine Learning.

In machine learning, models often predict *unnormalized log probabilities*. These must often be converted into regular probabilities.

In this lab, you will explore the log-sum-exp function, which is described in the text (Sec. 2.5.4).  You will code up several variants of the function, and compare their performance.

# Part 1: Logsumexp
---
## Setup: The Iris Dataset
We'll begin by downloading the Iris dataset. The iris dataset is a simple, but very famous, dataset introduced to the world by RA Fisher (the “father” of modern statistics”) in 1939. The dataset has five columns:
* sepal length (cm)
* sepal width (cm)
* petal length (cm)
* petal width (cm)
* class

In order to get logits to play with, we'll first train a multinomial logistic regression model (Sec. 2.5.3).  This model naturally outputs logits.

In [1]:
import datasets
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

ds = datasets.load_dataset( "scikit-learn/iris" )

df = pd.DataFrame( ds['train'] )

X = np.array( df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']] )
Y = np.array( LabelEncoder().fit_transform( df['Species'] ) )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression().fit(X,Y)

W = model.coef_
b = model.intercept_

b = np.reshape( b, (3,1))

logits = np.dot( W, X.T ) + b

print(logits.shape)

(3, 150)


---
## Exercise 1: convert logits to probabilities

Since our model outputs logits, they must be converted. To do this, we'll use the softmax function.

In [3]:
def softmax(logits):
  # logits is a numpy matrix of d x N, where:
  # d is the number of classes
  # N is the number of samples

  logits_copy = logits.copy()
  d, N = logits_copy.shape

  for sample in range(N):
    sum_exp = 0

    # Exponentiate the logits and sum them together for each row
    for data_class in range(d):
      logits_copy[data_class, sample] = np.exp( logits_copy[data_class, sample] )
      sum_exp += logits_copy[data_class, sample]

    # Divide each exponential by the row's sum
    for data_class in range(d):
      logits_copy[data_class, sample] = logits_copy[data_class, sample] / sum_exp

  return logits_copy

In [4]:
# print out test cases
probs = softmax(logits)
print (probs[:,0])
print (probs[:,120])

[9.81747643e-01 1.82523424e-02 1.43429404e-08]
[5.54234723e-06 2.39030160e-02 9.76091442e-01]


### test cases
probs = softmax( logits )

probs[:,0]
#### array([9.81803910e-01, 1.81960759e-02, 1.43430317e-08])
probs[:,120]
#### array([5.49519371e-06, 2.38812718e-02, 9.76113233e-01])

---
## Exercise 2: convert logits to probabilities

Now, code up the logsumexp function.  What test cases should you use for this function?

In [5]:
def logsumexp(logits):
  # logits is a numpy matrix of d x N, where:
  # d is the number of classes
  # N is the number of samples

  d, N = logits.shape
  result = np.zeros(d)

  for data_class in range(d):
    class_max = logits[data_class, 0]
    exp_sum = 0

    # Find the class max
    for sample in range(1, N):
      if logits[data_class, sample] > class_max:
        class_max = logits[data_class, sample]

    # Subtract class max, then exponentiate and sum
    for sample in range(N):
      exp_sum += np.exp(logits[data_class, sample] - class_max)

    # Save log(sum) + class_max
    result[data_class] = np.log(exp_sum) + class_max

  return result

In [6]:
# test cases
print (logsumexp(logits))

[11.15016899  7.69932095  9.8202921 ]


What should be printed??

I'm not sure I understand the question, but each value returned by logsumexp(logits) is the weighted sum of every value for that class. A larger number means that the given class is more prevalent, but it's not a clear ratio like softmax.

---
## Exercise 3: explore underflow / overlow

First, code up a function that compares two distributions. This can be anything you want; you may consider things like the MSE.

In [7]:
def compare_probs( probs1, probs2 ):
  # Normalize probs2 to sum to 1 (convert logsumexp "scores" into probabilities)
  probs2_norm = np.exp(probs2) / np.sum(np.exp(probs2))

  # Take mean of probs1 across flowers to get one distribution per class
  probs1_mean = np.mean(probs1, axis=1)

  # Compute Mean Squared Error
  mse = np.mean((probs1_mean - probs2_norm)**2)

  return mse

In [8]:
probs1 = softmax( logits )
probs2 = logsumexp( logits )
print(compare_probs( probs1, probs2 ))

0.10135992344225586


Now, see what happens if you add (or subtract) a constant from logits. How big must the constant be before things start going haywire?

In [9]:
# your code here
C_values = [0, 100, 500, 700, 1000]

for C in C_values:
  probs = softmax(logits + C)

  # Detect overflow or NaNs
  if np.any(np.isnan(probs)) or np.any(np.isinf(probs)):
    print(f"C = {C} caused overflow!")
    break

  else:
    print(f"C = {C}, probs[:,0] = {probs[:,0]}")

C = 0, probs[:,0] = [9.81747643e-01 1.82523424e-02 1.43429404e-08]
C = 100, probs[:,0] = [9.81747643e-01 1.82523424e-02 1.43429404e-08]
C = 500, probs[:,0] = [9.81747643e-01 1.82523424e-02 1.43429404e-08]
C = 700, probs[:,0] = [9.81747643e-01 1.82523424e-02 1.43429404e-08]
C = 1000 caused overflow!


/tmp/ipython-input-25308393.py:14: RuntimeWarning: overflow encountered in exp
  logits_copy[data_class, sample] = np.exp( logits_copy[data_class, sample] )
/tmp/ipython-input-25308393.py:19: RuntimeWarning: invalid value encountered in scalar divide
  logits_copy[data_class, sample] = logits_copy[data_class, sample] / sum_exp


Now convert the logits to 16-bit precision, and re-run your experiments. Analyze the differences you see (2-3 sentences).

In [10]:
logits16 = logits.astype( np.float16 )

# your code here
for C in C_values:
  probs = softmax(logits16 + C)

  # Detect overflow or NaNs
  if np.any(np.isnan(probs)) or np.any(np.isinf(probs)):
    print(f"C = {C} caused overflow!")
    break

  else:
    print(f"C = {C}, probs[:,0] = {probs[:,0]}")

C = 0, probs[:,0] = [0.9814  0.01822 0.     ]
C = 100 caused overflow!


/tmp/ipython-input-25308393.py:14: RuntimeWarning: overflow encountered in exp
  logits_copy[data_class, sample] = np.exp( logits_copy[data_class, sample] )
/tmp/ipython-input-25308393.py:19: RuntimeWarning: invalid value encountered in scalar divide
  logits_copy[data_class, sample] = logits_copy[data_class, sample] / sum_exp


### Analysis

Not surprisingly, the lower-bit structure overflowed sooner than the higher-bit structure, by a significant margin. I'm not really sure what the take away is supposed to be beyond that...

---
## Exercise 4: cleanly compute log probabilities

Sometimes, we want to compute log probabilities (which are different from logits), but we want to do so "cleanly", ie, while avoiding overflow / underflow. First, mathematically figure out what the log of the softmax is (ie, take the log of eq. 2.99), and then combine it with insights from coding up the logsumexp function. Hint: at the end of the day, you will simply shift each column by a per-column constant!

In [11]:
def log_logsumexp( logits ):
  # logits is a numpy matrix of d x N, where:
  # d is the number of classes
  # N is the number of samples

  LSE = logsumexp(logits)
  log_softmax = logits.copy()
  d, N = log_softmax.shape

  for data_class in range(d):
    for sample in range(N):
      log_softmax[data_class, sample] = log_softmax[data_class, sample] - LSE[data_class]

  return log_softmax


In [12]:
print(log_logsumexp(logits)[:,0])

[ -3.8079597   -4.34215254 -20.5196698 ]


---
# Part 2: Probability Fundamentals

For the following exercises, you are encouraged to work both by hand and by code, whatever makes the most sense.



## Exercise 1a: Joint Probability Distributions

You are given the following two binary variables, X and Y, that can each take on the values 0 or 1. Assuming X and Y are independent, calculate the joint probability table (2x2 table for P(X, Y)). Display as a numpy array.

P(X=0) = 0.6

P(X=1) = 0.4

P(Y=0) = 0.7

P(Y=1) = 0.3


In [13]:
X = [.6, .4]
Y = [.7, .3]

In [14]:
# Your code here
joint_prob = np.array([[X[0]*Y[0], X[0]*Y[1]], [X[1]*Y[0], X[1]*Y[1]]])
print(joint_prob)

[[0.42 0.18]
 [0.28 0.12]]


Next, compute the following conditional probabilities:

P(X=0|Y=0) = ?

P(X=0|Y=1) = ?

P(Y=0|X=0) = ?

P(Y=0|X=1) = ?

In [15]:
# Your code here
def con_prob(event, given):
    return round((event / given), 3)

px = joint_prob.sum(axis=1)
py = joint_prob.sum(axis=0)

first_x0y0 = con_prob(joint_prob[0,0], py[0])
first_x0y1 = con_prob(joint_prob[0,1], py[1])
first_y0x0 = con_prob(joint_prob[0,0], px[0])
first_y0x1 = con_prob(joint_prob[1,0], px[1])

print("P(X=0|Y=0) =", first_x0y0)
print("P(X=0|Y=1) =", first_x0y1)
print("P(Y=0|X=0) =", first_y0x0)
print("P(Y=0|X=1) =", first_y0x1)

P(X=0|Y=0) = 0.6
P(X=0|Y=1) = 0.6
P(Y=0|X=0) = 0.7
P(Y=0|X=1) = 0.7


Compare the result of these conditional probabilities to the original marginal probabilities given. What does this say about the relationship between variable dependence and using conditional probabilities? Write 1-2 sentences.

P(X=0) = 0.6, regardless of Y. Likewise, P(Y=0) = 0.7, regardless of X. This means that the variables are independent of each other, which further means that knowing one provides no information about the other.

## Exercise 1b: Joint Probability Distributions

Now consider this joint distribution:

|  | $Y = 0$ | $Y = 1$|
| :------- | :------: | -------: |
| $X = 0$  | 0.45  | 0.10  |
| $X = 1$  | 0.25  | 0.20  |

First, compute the marginals from the joint table

In [16]:
joint_prob = np.array([[0.45, 0.10], [0.25, 0.20]])

# P(X=0), P(X=1) → sum across Y (axis=1)
px = joint_prob.sum(axis=1)

# P(Y=0), P(Y=1) → sum across X (axis=0)
py = joint_prob.sum(axis=0)

print("P(X=0) =", round(px[0], 3))
print("P(X=1) =", round(px[1], 3))
print("P(Y=0) =", round(py[0], 3))
print("P(Y=1) =", round(py[1], 3))


P(X=0) = 0.55
P(X=1) = 0.45
P(Y=0) = 0.7
P(Y=1) = 0.3


Compute the same conditional probabilities as above:

P(X=0|Y=0) = ?

P(X=0|Y=1) = ?

P(Y=0|X=0) = ?

P(Y=0|X=1) = ?

In [17]:
second_x0y0 = con_prob(joint_prob[0,0], py[0])
second_x0y1 = con_prob(joint_prob[0,1], py[1])
second_y0x0 = con_prob(joint_prob[0,0], px[0])
second_y0x1 = con_prob(joint_prob[1,0], px[1])

print("P(X=0|Y=0) =", second_x0y0)
print("P(X=0|Y=1) =", second_x0y1)
print("P(Y=0|X=0) =", second_y0x0)
print("P(Y=0|X=1) =", second_y0x1)


P(X=0|Y=0) = 0.643
P(X=0|Y=1) = 0.333
P(Y=0|X=0) = 0.818
P(Y=0|X=1) = 0.556


Check if the independence property $P(X, Y) = P(X)P(Y)$ holds for any cell.

In [18]:
# Your answer here
if second_x0y0 == px[0] * py[0]:
  print("X = 0 and Y = 0 are independent")
else:
  print("X = 0 and Y = 0 are not independent")

if second_x0y1 == px[0] * py[1]:
  print("X = 0 and Y = 1 are independent")
else:
  print("X = 0 and Y = 1 are not independent")

if second_y0x0 == py[0] * px[0]:
  print("Y = 0 and X = 0 are independent")
else:
  print("Y = 0 and X = 0 are not independent")

if second_y0x1 == py[0] * px[1]:
  print("Y = 0 and X = 1 are independent")
else:
  print("Y = 0 and X = 1 are not independent")

X = 0 and Y = 0 are not independent
X = 0 and Y = 1 are not independent
Y = 0 and X = 0 are not independent
Y = 0 and X = 1 are not independent


Compare P(X=0|Y=0) to P(X=0|Y=1), and discuss what this says about the dependence between these variables (1-2 sentences).

P(X=0|Y=0) != P(X=0|Y=1), meaning that the variables are not independent of each other, meaning that if you know something about one, you know something about the other.

<br>

---
## Exercise 2: Bayes Theorem

After your yearly checkup, the doctor has bad news and good news. The bad news is that you tested positive
for a serious disease, and that the test is 99% accurate (i.e., the probability of testing positive given that you
have the disease is 0.99, as is the probability of testing negative given that you don’t have the disease). The
good news is that this is a rare disease, striking only one in 10,000 people. What are the chances that you
actually have the disease? (Show your calculations as well as giving the final result.)

*Hint: write out the variables you know, and think about what you'll need to calculate to find the final answer

In [19]:
test_accuracy = 0.99
diseased = 1 / 10000

diseased_given_positive = (test_accuracy * diseased) / ((test_accuracy * diseased) + ((1 - test_accuracy) * (1 - diseased)))
print(str(round((diseased_given_positive * 100), 3)) + "%")

0.98%
